In [1]:
import os
from PIL import Image

input_folder = "images/xfx_512_512"
output_folder = "images/xfx_512_512_resized"

os.makedirs(output_folder, exist_ok=True)

for filename in os.listdir(input_folder):
    if filename.lower().endswith((".png", ".jpg", ".jpeg", ".webp")):
        input_path = os.path.join(input_folder, filename)
        output_path = os.path.join(output_folder, filename)

        image = Image.open(input_path).convert("RGB")
        image = image.resize((512, 512), Image.LANCZOS)

        image.save(output_path)

        print(f"Resized: {filename}")

print("Done.")

Resized: xlgfx56_01.jpg
Resized: xlgfx30_06.jpg
Resized: xlgfx54_05 - Copy.jpg
Resized: xlgfx52_07.jpg
Resized: xlgfx42_07.jpg
Resized: xlgfx20_08.jpg
Resized: xlgfx23_06.jpg
Resized: xlgfx53_02.jpg
Resized: xlgfx23_10.jpg
Resized: xlgfx19_08.jpg
Resized: xlgfx31_08.jpg
Resized: xlgfx49_06.jpg
Resized: xlgfx52_06.jpg
Resized: xlgfx38_05.jpg
Resized: xlgfx42_05.jpg
Resized: xlgfx08_13.jpg
Resized: xlgfx56_06.jpg
Resized: xlgfx13_09.jpg
Resized: xlgfx45_05.jpg
Resized: xlgfx52_05.jpg
Resized: xlgfx51_08.jpg
Resized: xlgfx08_03.jpg
Resized: xlgfx08_01.jpg
Resized: xlgfx56_04.jpg
Resized: xlgfx43_03.jpg
Done.


In [38]:
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import onnxruntime as ort
import pandas as pd
print("ONNX Runtime version:", ort.__version__)
ImagePath ='/home/aistudio/PytorchStableDiffusion/images/xfx_512_512'
WD14TaggerPath ='/home/aistudio/models/wd-v1-4-convnextv2-tagger-v2/model.onnx'
TagCSVPath = '/home/aistudio/models/wd-v1-4-convnextv2-tagger-v2/selected_tags.csv'
TriggerWords = 'xiaofeixiang'
session = ort.InferenceSession(WD14TaggerPath)
tags = pd.read_csv(TagCSVPath)
tagsList =tags['name'].tolist()
print(tagsList[:10])
fileList = os.listdir(ImagePath)
fileImageList = []
for fileName in fileList:
    if fileName.endswith('.jpg'):
        fileImageList.append(fileName)
fileList = fileImageList
print(fileList)
for fileName in fileList:    
    inputPath = os.path.join(ImagePath, fileName)
    print(inputPath)
    image = cv2.imread(inputPath)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = image.astype(np.float32)
    image = cv2.resize(image, (448, 448))
    #plt.imshow(image/255.0)
    #plt.show()
    #image = image.transpose((2, 0, 1))
    image = np.expand_dims(image, axis=0)
    #print(image, image.shape)
    inputName = session.get_inputs()[0].name
    outPut = session.run(None,{inputName:image})
    print(type(outPut), len(outPut))
    print(outPut[0].shape)
    outPutIndexList =outPut[0][0]
    print(outPutIndexList)
    tagOutTextList = []
    for i,tagConfidene in enumerate(outPutIndexList):
        if tagConfidene > 0.2:
            tagOutTextList.append(tagsList[i].replace('_',' '))
    tagStr = TriggerWords
    for tag in tagOutTextList:
        tagStr = tagStr + ',' + tag       
    textPath = inputPath.split('.')[0] + '.txt'
    with open(textPath, 'w') as f:
        f.write(tagStr)
    print(tagStr)


ONNX Runtime version: 1.23.2
['general', 'sensitive', 'questionable', 'explicit', '1girl', 'solo', 'long_hair', 'breasts', 'looking_at_viewer', 'blush']
['xlgfx56_04.jpg', 'xlgfx43_03.jpg', 'xlgfx56_01.jpg', 'xlgfx45_05.jpg', 'xlgfx38_05.jpg', 'xlgfx49_06.jpg', 'xlgfx08_03.jpg', 'xlgfx52_06.jpg', 'xlgfx08_01.jpg', 'xlgfx54_05.jpg', 'xlgfx08_13.jpg', 'xlgfx42_05.jpg', 'xlgfx56_06.jpg', 'xlgfx23_10.jpg', 'xlgfx20_08.jpg', 'xlgfx42_07.jpg', 'xlgfx23_06.jpg', 'xlgfx19_08.jpg', 'xlgfx52_07.jpg', 'xlgfx13_09.jpg', 'xlgfx52_05.jpg', 'xlgfx51_08.jpg', 'xlgfx31_08.jpg', 'xlgfx53_02.jpg', 'xlgfx30_06.jpg']
/home/aistudio/PytorchStableDiffusion/images/xfx_512_512/xlgfx56_04.jpg
<class 'list'> 1
(1, 9083)
[6.8181598e-01 3.0335802e-01 2.7686059e-03 ... 1.1354685e-05 1.3411045e-06
 1.3113022e-06]
xiaofeixiang,general,sensitive,1girl,solo,long hair,looking at viewer,smile,bangs,shirt,black hair,bow,animal ears,school uniform,upper body,outdoors,teeth,cat ears,bowtie,grin,black eyes,colored skin,portr

In [ ]:

import torch
from torch.utils.data import Dataset

class LoraDataSet(Dataset):
    def __init__(self,filePath):
        super(LoraDataSet,self).__init__()
        pass
    
    
    def __getitem__(self,index):
        pass 
    def __len__(self):
        pass
        
    